In [2]:
# sudo -H anaconda/bin/pip install biopython

from Bio import Entrez

In [ ]:
Entrez.email = 'greg.clinton@gmail.com'

In [16]:
record = Entrez.read(Entrez.esearch(db = 'gene', term = 'Cypripedioideae[Orgn] AND matK[Gene]'))
record['IdList']

['19737447', '26118297', '24019622', '24019231', '24018806', '24571736']

In [21]:
record = Entrez.read(Entrez.efetch(db = 'gene', id = '19737447', retmode = 'xml'))

In [6]:
Entrez.read(Entrez.einfo())['DbList']

['pubmed', 'protein', 'nuccore', 'nucleotide', 'nucgss', 'nucest', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'clone', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosystems', 'pccompound', 'pcsubstance', 'pubmedhealth', 'seqannot', 'snp', 'sra', 'taxonomy', 'unigene', 'gencoll', 'gtr']

In [8]:
record = Entrez.read(Entrez.egquery(term = 'biopython'))

for row in record['eGQueryResult']:
    if int(row['Count']) > 0:
        print row['DbName'], row['Count']

pubmed 21
pmc 644
books 2
pubmedhealth 2
gds 16


In [11]:
record = Entrez.read(Entrez.einfo(db = 'gene'))

for field in record['DbInfo']['FieldList']:
    print '%(Name)s, %(FullName)s, %(Description)s' % field

ALL, All Fields, All terms from all searchable fields
UID, UID, Unique number assigned to a gene record
FILT, Filter, Limits the records
TITL, Gene/Protein Name, gene or protein name
WORD, Text Word, Free text associated with record
ORGN, Organism, scientific and common names of organism
MDAT, Modification Date, The last date on which the record was updated
CHR, Chromosome, Chromosome number or numbers; also 'mitochondrial', 'unknown' properties
MV, Default Map Location, Chromosomal map location as displayed in MapViewer
GENE, Gene Name, Symbol or symbols of the gene
ECNO, EC/RN Number, EC number for enzyme or CAS registry number
MIM, MIM ID, MIM number from OMIM
DIS, Disease/Phenotype, Name(s) of diseases associated with this gene. When available, OMIM name will be used
ACCN, Nucleotide/Protein Accession, Nucleotide or protein accession(s) associated with this gene
UGEN, UniGene Cluster Number, UniGene cluster number for this gene
PROP, Properties, Properties of Gene record
CDAT, Crea